# Tutorial 3: Loops and Conditionals


# Introduction

In lecture Tuesday we learned that combinations of conditionals and loops could make our code much more powerful. In last week’s tutorial, you loaded up a spectrum, performed some basic array manipulation and slicing on it, and viewed it. Today we are going to continue with spectral analysis, this time performing a step known as ”centroiding”. The code you will need to write to load up, correct, and centroid a spectrum will require you to use both loops and conditionals.


# Spectral Centroiding and Calibration

If we take a look at a typical a neon emission spectrum, (or any gas for that matter) we would see sharp emission peaks at very distinct wavelengths. This occurs because when atoms are energized (in the case of a neon lamp, by an electric current, and in the case of gas in space, by high energy photons coming from elsewhere) they release photons, but they can only do so at several quantized energies (wavelengths). Think of the Bohr model of the atom. An energized electron can drop from the 4th to the 2nd energy level, or the 3rd to 2nd, or the 2nd to 1st, etc. Because those energy levels are quantized, the energies of the photons emitted when an electron falls n levels is also quantized. Every atom has a specific arrangement of electron orbitals that makes it such that the combination of wavelengths it can output due to electron cascading is unique (so a trained spectroscopist can look at almost any emission spectrum and recognize the element!). 

However, when we view spectra like the one we will plot below, we notice that the sharp peaks do in fact have a certain width, despite the description above indicating they should be perfect vertical lines at some defined wavelengths. There are numerous reasons for the widening: spectrometers introduce some widening (instrumental effects- better spectrometers have smaller widths), fast moving gas has doppler shifts associated with some photons, and, at a theoretical level, we know from quantum physics that there is always imprecision in either the electron's position or momentum (though this would be a tiny effect in this situation). 

In any case, if we are to perform any sort of scientific analysis on a spectrum, we need to know for each of those peaks what our best guess for the single "right" wavelength for that line should be. One typical method of determining this is centroiding.

**What we will learn** In addition to the basics of actually centroiding a spectrum, this tutorial is meant as an introduction to loops and conditionals, the building blocks of most python code. 


## Quick Background

Centroiding amounts to finding the "center of mass" of a given peak. The formula for determining the centroid of a range of values (such as intensity/signal) is
\begin{equation}
x_{cm} = \frac{\sum x_iI_i}{\sum I_i}.
\end{equation}
where $x_i$ will end up being our wavelength (or pixel number) array, and "I" represents intensity (or Brightness, or # of photons/unit time) at each pixel value. It basically represents a "weighted average" of the peak. Note: we can't simply choose the x value where the peak hits its maximum because the limitations of the spectrometer result in the true "peak value" not lining up with the real wavelength we are looking for. This weighted average of the whole peak provides a better guess. Think about it, just because one extra photon happens to pile into pixel x, doesn't mean pixel x is representitive of that spectral line.

Additionally, spectrometers tend to introduce a characteristic "shape" to peaks, unique to that spectrometer. We want to get away from those effects and treat each peak in statistically the same way.

In [ ]:
####### ALWAYS RUN THIS CELL!!!!!!!!!

import numpy as np 
import matplotlib.pyplot as plt
!pip install -U okpy 
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)

from client.api.notebook import Notebook
ok = Notebook('hw3.ok')

Now we need to load the data about the neon spectrum we'll be centroiding into our program. You'll remember that last week the data were stored in a FITS file which we read in. This time, our data are stored in a simple text file ("neon.txt"). Since neon.txt is sitting in our current directory, we can use the notebook's ability to run terminal commands to see what it looks like:

In [ ]:
!head neon.txt

The head command is an example of a useul UNIX command we didn't cover in lecture. ```head``` is the command which lets you peek into a file (so long as it is text readable) and output the first few lines (in case you don't want the whole thing to print to screen). Similar commands are "more", which opens a dialogue showing the first few lines and lets you enter or space (to page down) through the whole file (q to quit), or "cat" which pulls the whole file contents to the screen (also quit out with q). 

We can see by peaking at our neon.txt that it is a simple file with two columns. Some files will have a row at the top designated to telling you what each column represents. Here that's not the case, so I'll tell you the zeroth column (to use python indexing syntax) is pixel number on our spectrometer, and the first column is the flux (think, number of photons/unit time) received in each pixel (which you can think of like a photon bucket). 

We are going to use the numpy function ```np.loadtxt``` to read this data in. We'll talk about data I/O (in/out) next week, so we'll provide the syntax for the command and load the file for you (run the cell below).

In [ ]:
#Load up neon spectrum 
pixels = np.loadtxt('neon.txt',usecols=(0,)) #take zeroth column
signal = np.loadtxt('neon.txt',usecols=(1,)) #take first column

So now we have a pixel and signal array. Our goal is to find the centroid of all the peaks in the plot. Speaking of, let's plot pixels vs signal just to get an idea of what were are looking at:

In [ ]:
... #enter plotting command here, then run cell to see if it matches mine.
plt.xlabel('Pixel Value')
plt.ylabel('Signal')
plt.show()

Notice there are quite a few jitters there, and peaks that are very tiny. We are only interested in the major features, so when we are performing the following analysis, it would probably be smart to implement some sort of threshold to catch only the major peaks. Let's use 25,000.

In order to find the centroid of each peak, we need to first determine the peaks in the first place. Let's go ahead and write a peak-finding algorithm. There is more than one way to do this, but we'll focus on the following method: 

1. Iterate over a range that's the length of the pixel array (or signal, they're the same length by construction) checking whether each value of signal is higher or lower than the value of signal one to the left and one to the right, and greater than the threshold we've chosen (again, 25,000).
2. Check whether it is in fact higher than its two left and right neighbors. 
3. If it is, call it a peak and append that index where the peak occurs out into a separate array.

I've started the loop for you, inside a function:

In [ ]:
#Find the peaks 
def peak_finder(signal_array):
    threshold = ...
    peaks = []                            #x positions of the peaks, or rather, their index
    for i in range(2,len(signal)-2): 
        if ___ and ___ and ___ and ____:  #four conditions to be a peak (see description)
            if ____:                      #is the value of the spectrum at i higher than our threshold?
                peaks.append(___)
    return peaks
    
#let's test our peaks function before running the ok test:
peaks = ...

In [ ]:
_ = ok.grade('q01')
_ = ok.backup()

**IN the cell here, write in an explanation for why the loop i initialized goes from range(2,len(signal)-2)**

We should now have the positions (in pixel space) where the peaks occur. We now want to calculate the centroid in a region around each peak. This raises a bit of an issue- how wide do we make that region? Too narrow and we don't include the whole peak, too wide and we are biased or influenced by things not actually a part of the peak. 

Turns out, most astronomers' solution to this issue is to use a standardized region known as the Full Width (at) Half Maximum (or FWHM). What this means is we want to use a region that is as wide as where the peak has dropped to half its maximum value. This works well, because the width will automatically be adjusted to handle larger and smaller peaks, and the same "amount" of each peak is being used in our calculation. 


In the space below, iterate over the peaks we just found, and determine the FWHM (in this case it will be very narrow because our spectrometer did not have very high resolution). I suggest defining an xmin and xmax, and using the "np.where" function to search out where the signal array drops below the half max value (easily calculated by taking signal indexed at "i" and dividing by 2). Using that FWHM, caculate the center of mass of each peak, and append the final pixel positions of the centers of mass to a new array/list. Remember that you can sum arrays (and arrays times arrays) without iterating!

In [ ]:
def centroid(x_range,y_range):
    '''A function to return the centroid given equally sized x and y ranges over which to perform the calculation'''
    x_range = np.array(x_range) #make sure these are arrays if they aren't already
    y_range = np.array(y_range) #make sure these are arrays if they aren't already
    ... #convert the math formula for a centroid into code in these lines
    ...
    x_centroid = ...
    return x_centroid

def find_all_centroids(pixels,signal):
    peaks = ....      #call our peak finding function
    centroids = []    #Values for all the centroids, to be filled by the for-loop below
    for i in ... :
        index_range = 5                  #lets index at a 5 pixel range around the peak
        centroid_out = centroid(...,...) #calling our new centroid function on specific parts of the neon spectrum
        centroids.append(...)            #put all the centroid values into the empty the centroids list
    return np.array(centroids)

#Again, lets test our function to see what it outputs, before running the ok test:
centroids = ...
print(centroids) #You have to have done the peakfinding right to get this answer right

In [ ]:
_ = ok.grade('q02')
_ = ok.backup()


Awesome! Now we have our centroids. Lets go ahead and plot the centroids as vertical, dashed lines over our data. As this is a somewhat trivial syntax lookup, I've just gone ahead and included it here: 

In [ ]:
centroids = find_all_centroids(pixels,signal) #call our centroid function
def plot_vert(x): 
    '''
    Just plots vertical lines, in blue dashes
    '''
    plt.axvline(x, color='blue', ls='-.')
    
for i in centroids[1:]: #Call my plotting function on every centroid except the first
    plot_vert(i)
    
plt.axvline(centroids[0],color='blue',ls='-.',label='Centroid') #Reserve the first so I don't have a million "centroid" labels
plt.plot(pixels, signal, 'r', label='Spectrum') #Plot the actual spectrum
plt.xlabel('Pixel Value')
plt.ylabel('Signal')
plt.legend(loc=2)
plt.show() #Show it

Looks like we've done this correctly; the vertical lines line up with the peaks of the spectrum, roughly (they shouldn't be at the exact peak, if we calculated our centers of mass correctly. In fact, let's plot this just to show ourselves that we did in fact get something different. We still have our "peaks" array, which contains the pixel positions of the peak values. Let's plot a "residual" (not exactly a residual in a scientific context, but close enough)- plot the difference between the peaks array and the centroids array against a new x-array that is just a range the length of the peaks/centroids array, to get a feel for whether the centroids were actually any different than the peaks:

In [ ]:
residual = ...
plt.plot(np.arange(len(residual)),residual,'bo')
plt.xlabel('Peak number (of those found)')
plt.ylabel('Residual [peak value - centroid value]')
plt.axhline(0,ls='-.')
plt.show()

We can see in the plot above several things, the first being that there is definitely a difference between the peak value and the centroid value, sometimes by as much as 3 pixels (this is of course specific to this spectrum taken with this spectrometer- values will differ). Secondly, we ca see that they seem to be randomly distributed, but perhaps slightly weighted to being on one side of zero. Assuming no systematics (of which there are likely several here), we would expect roughly half the points to be negative (roughly have the centroids be to the left of the peak and half to the right). 

We would want to, as scientists, explore further why this bias might exist. Is there bias in how we are calculating our FWHM (perhaps accidentally weighting to the right? Perhaps the spectrograph itself has a slight systematic bias towards shuffling incoming photons into slightly higher wavelength bins than they should be? Perhaps, in some inherent sense, the shape of neon spectral peaks is not symmetric but is in fact slightly asymmetric?)

Good questions to ask and interrogate, and which will be left as an exercise to the reader (I've always wanted to say that!)

## Wait, what the hell do I do with these centroids?

You might be asking "What do I do now, with the centroids?" Often, your next step would be calibrating your spectrometer. Notice our x-axis units have been pixels- that is, there is a CCD (charge coupled device) in the spectrometer, and a diffraction grating (acting the way a prism does) to deflect light of different wavelengths/energies onto different pixels in the CCD. We would of course rather like to know what **wavelengths** these emission lines correspond to. Thus, we need to know the pixel to wavelength ratio of our spectrometer. By recognizing certain lines (a skill invaluable to spectroscopists and that often involves squinting at other spectra looking for similarities in pattern), we can map our 20 ish calculated centroids to 20 ish known wavelengths for those lines (from a database like NIST), and through a fitting proccess determine the formula to then turn any pixel in our ccd into a wavelength. We'll get to fitting data soon!

# BONUS POINTS

**Option to Earn some Bonus points with an extra function! Wanna be more scientific???? READ ON!!!!!**

## Full Width at Half Maximum

In the centroiding code above, we took each peak and found the center of mass in a region of five pixels around each peak. But looking at our plots, we can see that some peaks are wider than others. We want to have a more statistically consistent method of taking the same percentage of the light of each peak when calculating our centroid. In comes the full with half maximum. 

The Full Width at Half Maximum (or FWHM) is the width of the peak at half the maximum intensity. (see below). 

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://ned.ipac.caltech.edu/level5/Leo/Figures/figure4.jpeg")

Write a function below (we've named it for you) that has the same functionality as ```find_all_centroids``` but instead of centroiding the 5 pixels around each peak, it centroids around the FWHM of the peak. This is a little bit tricky. You will need the function ```np.where()``` -- look it up and see if you can understand how it works. (also as a note, in any place you call ```np.where()``` immediately index it at 0 (that is, ```np.where(blah)[0]```) because it annoyingly returns a tuple for which we only want the first index. This function should call the same centroid function you wrote above. You can pull in the same peaks you found earlier. 

In [ ]:
def find_all_centroids_with_FWHM(pixels,signal):
    peaks = peak_finder(signal) #read in our peaks using peak_finder function
    ...
    ...
     .
     .
     .
    ...
    return np.array(centroids)


In [ ]:
_ = ok.grade('qEC')
_ = ok.backup()

In [ ]:
# Now, we'll submit to okpy
_ = ok.submit()